Import the pynrrd package supplementary class. Load the image and mask.

In [1]:
import nrrd
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

# paths to image and corresponding mask
im_path = 'data/images/LIDC-IDRI-0001.nrrd'
msk_path = 'data/masks/LIDC-IDRI-0001.nrrd'

im_V,im_d = nrrd.read(im_path)
msk_V,msk_d = nrrd.read(msk_path)


Bad key "text.kerning_factor" on line 4 in
/Users/EL-CAPITAN-2016/anaconda2/envs/spyder/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


Use `skimage.measure`'s `label` method to locate the radiologist-defined mask. Recall that each pixel in the mask was agreed to be in the lesion with minimum 50% agreement between 4 radiologists.

**WILL NEED TO ADJUST SCRIPT FOR >1 LESION PER SCAN**

In [2]:
from skimage.measure import label,regionprops

# label the mask into connected regions
nodule_labels,num_labels = label(msk_V,return_num=True)
nodule_props = regionprops(nodule_labels)

# the number of nodules in nodule_props corresponds to the number of connected regions
print('There are {} nodule(s) in this image.'.format(num_labels))     

There are 1 nodule(s) in this image.


In [3]:
# isolate the centroid of the nodule -- this will be our marker for the watershed algorithm
nodule_props[0].centroid
centroid = [round(i) for i in nodule_props[0].centroid] 
centroid

[367, 316, 90]

In [4]:
from skimage.segmentation import watershed

# watershed?

In [5]:
from scipy.ndimage import median_filter as median

# denoise image
denoised = median(im_V,3)

print(denoised.shape)
isv(denoised)

(512, 512, 133)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [6]:
from skimage.morphology import erosion, dilation, ball

# try some funky stuff
max_image = dilation(denoised,ball(2))
min_image = erosion(denoised,ball(2))

gradient_image = max_image - min_image

isv(gradient_image)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [78]:
from skimage.measure import label
from skimage.morphology import binary_opening

threshold_gradient = gradient_image < 350 
isv(threshold_gradient)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [70]:
import matplotlib.pyplot as plt

markers = label(threshold_gradient)
isv(markers,cmap='magma')

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [71]:
# process the watershed
labels = watershed(gradient_image, markers)

In [72]:
isv(labels,cmap='magma')

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [73]:
labels.shape
centroid
water_mask = labels == labels[centroid[0],centroid[1],centroid[2]]

In [12]:
isv(water_mask)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [36]:
from skimage.measure import regionprops
from skimage.morphology import binary_opening

mask_props = label(water_mask)
mask_attr = regionprops(mask_props)
# mask_attr[0].major_axis_length

radius = mask_attr[0].major_axis_length / 2 * 0.03

water_mask_clean = binary_opening(water_mask,ball(radius))

In [37]:
isv(water_mask_clean)      # <-- doesn't seem to work well

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [74]:
import numpy as np

iou_mask = np.count_nonzero(np.logical_and(water_mask,msk_V))/np.count_nonzero(np.logical_or(water_mask,msk_V))
iou_back = np.count_nonzero(np.logical_and(~water_mask,~msk_V))/np.count_nonzero(np.logical_or(~water_mask,~msk_V))

In [75]:
iou_mask     # <-- IoU for the mask

0.7888789237668161

In [76]:
iou_back

0.9998696406084792

In [77]:
(iou_mask + iou_back) / 2    # <-- mean IoU

0.8943742821876477

In [56]:
from skimage.segmentation import morphological_chan_vese

mask2 = morphological_chan_vese(denoised, 50)

In [57]:
isv(mask2)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…